# Automação Web e Busca de Informações do Dolar

In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# para rodar o chrome em 2º plano
# from selenium.webdriver.chrome.options import Options
# chrome_options = Options()
# chrome_options.headless = True 
# navegador = webdriver.Chrome(options=chrome_options)

# abrir um navegador
navegador = webdriver.Chrome()
# caso queira deixar na mesma pasta do seu código
# navegador = webdriver.Chrome("chromedriver.exe")


navegador.get("https://www.google.com/")

# Passo 1: Pegar a cotação do Dólar
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")

navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element(By.XPATH,
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value") 
print(cotacao_dolar)

# Passo 2: Pegar a cotação do Euro
navegador.get("https://www.google.com/")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH,
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_euro)

# Passo 3: Pegar a cotação do Ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)

navegador.quit()

5.2789
5.688278695
338.43


### Agora vamos atualizar a nossa base de preços com as novas cotações

- Importando a base de dados

In [10]:
# Passo 4: Importar a lista de produtos
import pandas as pd
tabela = pd.read_excel('Produtos.xlsx', index_col=0)

print(tabela)

                Preço Original  Moeda  Cotação  Preço de Compra  Margem  \
Produtos                                                                  
Câmera Canon            999.99  Dólar        5          4999.95    1.40   
Carro Renault          4500.00   Euro        6         27000.00    2.00   
Notebook Dell           899.99  Dólar        5          4499.95    1.70   
IPhone                  799.00  Dólar        5          3995.00    1.70   
Carro Fiat             3000.00   Euro        6         18000.00    1.90   
Celular Xiaomi          480.48  Dólar        5          2402.40    2.00   
Joia 20g                 20.00   Ouro      350          7000.00    1.15   

                Preço de Venda  
Produtos                        
Câmera Canon          6999.930  
Carro Renault        54000.000  
Notebook Dell         7649.915  
IPhone                6791.500  
Carro Fiat           34200.000  
Celular Xiaomi        4804.800  
Joia 20g              8050.000  


- Atualizando os preços e o cálculo do Preço Final

In [11]:
# Passo 5: Recalcular o preço de cada produto
# atualizar a cotação
# nas linhas onde na coluna "Moeda" = Dólar
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# atualizar o preço base reais (preço base original * cotação)
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# atualizar o preço final (preço base reais * Margem)
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

# tabela["Preço de Venda"] = tabela["Preço de Venda"].map("R${:.2f}".format)

display(tabela)

,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
Produtos,,,,,,
Câmera Canon,999.99,Dólar,5.278900,5278.847211,1.40,7390.386095
Carro Renault,4500.00,Euro,5.688279,25597.254128,2.00,51194.508255
Notebook Dell,899.99,Dólar,5.278900,4750.957211,1.70,8076.627259
IPhone,799.00,Dólar,5.278900,4217.841100,1.70,7170.329870
Carro Fiat,3000.00,Euro,5.688279,17064.836085,1.90,32423.188561
Celular Xiaomi,480.48,Dólar,5.278900,2536.405872,2.00,5072.811744
Joia 20g,20.00,Ouro,338.430000,6768.600000,1.15,7783.890000


### Agora vamos exportar a nova base de preços atualizada

In [12]:
# Passo 6: Salvar os novos preços dos produtos
tabela.to_excel("Produtos.xlsx", index=False)